In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import BatchNormalization,Activation,MaxPooling2D,GlobalAveragePooling2D,Dense,Flatten,ZeroPadding2D,AveragePooling2D,Input,Add
from keras.models import Model
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import tensorflow as tf
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [2]:
img_shape=(224,224)

In [27]:
def identity_block(layer,f,nfilters):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)
    

In [42]:
def Conv_block(layer,f,nfilters,s=2):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),strides=(s,s))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    Xshortcut=Conv2D(F3,(1,1),strides=(s,s))(Xshortcut)
    Xshortcut=BatchNormalization(axis=-1)(Xshortcut)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)


In [47]:
def Resnet50(img_shape=(224,224,3),numclasses=1000):
    img_inp=Input(shape=img_shape,dtype=tf.float32)
    X=ZeroPadding2D((3,3))(img_inp)
    X=Conv2D(64,(7,7),strides=(2,2))(X)
    X=BatchNormalization(axis=-1)(X)
    X=Activation('relu')(X)
    
    X=MaxPooling2D((3,3),strides=(2,2),padding='same')(X)
    X=Conv_block(X,3,[64,64,256],s=1)
    X=identity_block(X,3,[64,64,256])
    X=identity_block(X,3,[64,64,256])
    
    X=Conv_block(X,3,[128,128,512],s=2)
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    
    X=Conv_block(X,3,[256,256,1024],s=2)
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    
    X=Conv_block(X,3,[512,512,2048],s=2)
    X=identity_block(X,3,[512,512,2048])
    X=identity_block(X,3,[512,512,2048])
    
    X=AveragePooling2D((7,7))(X)
    
    X=Flatten()(X)
    X=Dense(numclasses,activation='softmax')(X)
    model=Model(inputs=img_inp,outputs=X)
    return(model)

In [48]:
model1=Resnet50()

In [49]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_12 (ZeroPadding2 (None, 230, 230, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_334 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_12[0][0]          
__________________________________________________________________________________________________
batch_normalization_332 (BatchN (None, 112, 112, 64) 256         conv2d_334[0][0]                 
__________________________________________________________________________________________________
activation

In [50]:
len(model1.layers)

177